# crspm

> Retrieve and process data from WRDS CRSP Monthly Stock File

Since this is a proprietary dataset, in the documentation below, I can not show any of the data that is being retrieved/generated (will show only column names).

In [ ]:
#| default_exp wrds.crspm

In [ ]:
#|exports
from __future__ import annotations
from typing import List

import pandas as pd
import numpy as np

import pandasmore as pdm
from finsets.wrds import wrds_api

In [ ]:
#| exports
PROVIDER = 'Wharton Research Data Services (WRDS)'
URL = 'https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-security-files/monthly-stock-file/'
LIBRARY = 'crsp'
TABLE = 'msf'
NAMES_TABLE = 'msenames'
DELIST_TABLE = 'msedelist'
FREQ = 'M'
MIN_YEAR = 1925
MAX_YEAR = None
ENTITY_ID_IN_RAW_DSET = 'permno'
ENTITY_ID_IN_CLEAN_DSET = 'permno'
TIME_VAR_IN_RAW_DSET = 'date'
TIME_VAR_IN_CLEAN_DSET = f'{FREQ}date'

In [ ]:
#| export
def list_all_vars() -> pd.DataFrame:
    "Collects names of all available variables from WRDS `{LIBRARY}.{TABLE}` and `{LIBRARY}.{NAMES_TABLE}`"

    try:
        db = wrds_api.Connection()
        msf = db.describe_table(LIBRARY,TABLE).assign(wrds_library=LIBRARY, wrds_table=TABLE)
        mse = db.describe_table(LIBRARY,NAMES_TABLE).assign(wrds_library=LIBRARY, wrds_table=NAMES_TABLE)
        dlst = db.describe_table(LIBRARY,DELIST_TABLE).assign(wrds_library=LIBRARY, wrds_table=DELIST_TABLE)
    finally:
        db.close()

    return pd.concat([msf, mse, dlst])[['name','type','wrds_library','wrds_table']].copy()

In [ ]:
#| eval: false
all_vars = list_all_vars()

Loading library list...
Done
Approximately 5153763 rows in crsp.msf.
Approximately 115366 rows in crsp.msenames.
Approximately 36809 rows in crsp.msedelist.


In [ ]:
#| eval: false
all_vars.name.count()

np.int64(61)

In [ ]:
#| eval: false
all_vars.head()

,name,type,wrds_library,wrds_table
0,cusip,VARCHAR(8),crsp,msf
1,permno,INTEGER,crsp,msf
2,permco,INTEGER,crsp,msf
3,issuno,INTEGER,crsp,msf
4,hexcd,SMALLINT,crsp,msf


In [ ]:
#| export
def default_raw_vars():
    """Defines default variables used in `get_raw_data` if none are specified."""
    
    return ['permno','permco','date',
            'ret', 'retx', 'shrout', 'prc', 
            'shrcd', 'exchcd','siccd','naics', 'ticker','cusip','ncusip',
            'dlret','dlstcd','dlstdt']            

In [ ]:
print(default_raw_vars())

['permno', 'permco', 'date', 'ret', 'retx', 'shrout', 'prc', 'shrcd', 'exchcd', 'siccd', 'naics', 'ticker', 'cusip', 'ncusip', 'dlret', 'dlstcd', 'dlstdt']


In [ ]:
#| export
def parse_varlist(vars: List[str]=None,
                  required_vars = [],
                  ) -> str:
    """Figures out which `vars` come from the `{LIBRARY}.{TABLE}` table and which come from the `{LIBRARY}.{NAMES_TABLE}` table and adds a. and b. prefixes to variable names to feed into an SQL query"""

    # Get all available variables and add suffixes needed for the SQL query
    suffix_mapping = {TABLE: 'a.', NAMES_TABLE: 'b.', DELIST_TABLE: 'c.'}
    all_avail_vars = list_all_vars().drop_duplicates(subset='name',keep='first').copy()
    all_avail_vars['w_prefix'] = all_avail_vars.apply(lambda row: suffix_mapping[row['wrds_table']] + row['name'] , axis=1)

    if vars == '*': return ','.join(list(all_avail_vars['w_prefix']))
    
    # Add required vars to requested vars
    if vars is None: vars = default_raw_vars()
    vars_to_get =  required_vars + [x for x in list(set(vars)) if x not in required_vars]

    # Validate variables to be downloaded (make sure that they are in the target database)
    invalid_vars = [v for v in vars_to_get if v not in list(all_avail_vars.name)]
    if invalid_vars: raise ValueError(f"These vars are not in the database: {invalid_vars}") 

    # Extract information on which variable comes from which wrds table, so we know what prefix to use
    vars_to_get = pd.DataFrame(vars_to_get, columns=['name'])
    get_these = vars_to_get.merge(all_avail_vars, how = 'left', on = 'name')
        
    return ','.join(list(get_these['w_prefix']))

In [ ]:
#| eval: false
parse_varlist(['ret', 'siccd', 'naics', 'dlret'])

Loading library list...
Done
Approximately 5153763 rows in crsp.msf.
Approximately 115366 rows in crsp.msenames.
Approximately 36809 rows in crsp.msedelist.


'b.naics,a.ret,c.dlret,b.siccd'

In [ ]:
#| eval: false
parse_varlist('*')

Loading library list...
Done
Approximately 5153763 rows in crsp.msf.
Approximately 115366 rows in crsp.msenames.
Approximately 36809 rows in crsp.msedelist.


'a.cusip,a.permno,a.permco,a.issuno,a.hexcd,a.hsiccd,a.date,a.bidlo,a.askhi,a.prc,a.vol,a.ret,a.bid,a.ask,a.shrout,a.cfacpr,a.cfacshr,a.altprc,a.spread,a.altprcdt,a.retx,b.namedt,b.nameendt,b.shrcd,b.exchcd,b.siccd,b.ncusip,b.ticker,b.comnam,b.shrcls,b.tsymbol,b.naics,b.primexch,b.trdstat,b.secstat,b.compno,c.dlstdt,c.dlstcd,c.nwperm,c.nwcomp,c.nextdt,c.dlamt,c.dlretx,c.dlprc,c.dlpdt,c.dlret,c.acperm,c.accomp'

In [ ]:
#| export
def get_raw_data(
        vars: List[str]=None, # If None, downloads `default_raw_vars`; use '*' to get all available variables
        required_vars = ['permno','date'], # Variables that are always downloaded, regardless `vars` argument
        nrows: int=None,  #Number of rows to download. If None, full dataset will be downloaded             
        start_date: str=None,          # Start date in MM/DD/YYYY format
        end_date: str=None,            # End date in MM/DD/YYYY format  
) -> pd.DataFrame:
    "Downloads `vars` from `start_date` to `end_date` from WRDS {LIBRARY}.{TABLE}, {LIBRARY}.{NAMES_TABLE} and {LIBRARY}.{DELIST_TABLE}." 

    wrds_api.validate_dates([start_date, end_date])
    varlist_string = parse_varlist(vars, required_vars=required_vars)
    sql_string = f"""SELECT {varlist_string}
                        FROM {LIBRARY}.{TABLE} AS a 
                        LEFT JOIN {LIBRARY}.{NAMES_TABLE} AS b
                            ON a.permno=b.permno AND b.namedt<=a.date AND a.date<=b.nameendt 
                        LEFT JOIN {LIBRARY}.{DELIST_TABLE} as c
                            ON a.permno=c.permno AND date_trunc('month', a.date) = date_trunc('month', c.dlstdt)
                            """
    sql_string += "WHERE 1=1 "
    if start_date is not None: sql_string += r" AND date >= %(start_date)s"
    if end_date is not None: sql_string += r" AND date <= %(end_date)s"
    if nrows is not None: sql_string += r" LIMIT %(nrows)s"

    df = wrds_api.download(sql_string,
                             params={'start_date':start_date, 'end_date':end_date, 'nrows':nrows})
    
    return df 

In [ ]:
#| eval: false
r = get_raw_data(vars = '*', nrows=100)
r.head(1)

Loading library list...
Done
Approximately 5153763 rows in crsp.msf.
Approximately 115366 rows in crsp.msenames.
Approximately 36809 rows in crsp.msedelist.
Loading library list...
Done


,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,nwperm,nwcomp,nextdt,dlamt,dlretx,dlprc,dlpdt,dlret,acperm,accomp
0,68391610,10000,7952,10396,3,3990,1985-12-31,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
#| export
def process_raw_data(
        df: pd.DataFrame=None,  # Must contain `permno` and `date` columns         
        clean_kwargs: dict={},  # Params to pass to `pdm.setup_panel` other than `panel_ids`, `time_var`, and `freq`
) -> pd.DataFrame:
    """Applies `pandasmore.setup_panel` to `df`"""

    # Change some variables to categorical
    for col in ['shrcd','exchcd']:
        if col in df.columns:
            df[col] = df[col].astype('Int64').astype('category')

    for col in ['naics','cusip','ncusip']:
        if col in df.columns:
            df[col] = df[col].astype('string').astype('category')

    if 'siccd' in df.columns:
        df['siccd'] = df['siccd'].astype('Int64').astype('string').str.zfill(4).astype('category')

    df = pdm.setup_panel(df, panel_ids=ENTITY_ID_IN_RAW_DSET, time_var=TIME_VAR_IN_RAW_DSET, freq=FREQ, 
                        panel_ids_toint=False, **clean_kwargs)
    return df 

In [ ]:
#| eval: false
df = process_raw_data(r)

In [ ]:
#| eval: false
df.head(0)

,,date,dtdate,cusip,permco,issuno,hexcd,hsiccd,bidlo,askhi,prc,...,nwperm,nwcomp,nextdt,dlamt,dlretx,dlprc,dlpdt,dlret,acperm,accomp
permno,Mdate,,,,,,,,,,,,,,,,,,,,,


In [ ]:
#| export
def delist_adj_ret(
        df: pd.DataFrame, # Requires `ret`,`exchcd`,`dlret`,`dlstcd`, and `dlstdt` variables
        adj_ret_var: str='ret_adj' # Name of the adjusted return variable created by this function
) -> pd.DataFrame:
    """Adjusts for returns for delisting using Shumway and Warther (1999) and Johnson and Zhao (2007)"""

    df['npdelist'] = (df['dlstcd']==500) | df['dlstcd'].between(520,584)
    
    cond12 = df['dlret'].isna() & df['npdelist'].fillna(False) & df['exchcd'].isin([1, 2])
    df.loc[cond12, 'dlret'] = -0.35
    
    cond3 = df['dlret'].isna() & df['npdelist'].fillna(False) & df['exchcd'].isin([3])
    df.loc[cond3, 'dlret'] = -0.55
    
    df.loc[df['dlret'].notna() & (df['dlret'] < -1), 'dlret'] = -1
    
    df['dlret'] = df['dlret'].fillna(0)

    df[adj_ret_var] = (1 + df.ret) * (1 + df['dlret']) - 1
    df[adj_ret_var] = np.where(df[adj_ret_var].isna() & (df['dlret']!=0), df['dlret'], df[adj_ret_var])
    df = df.drop('npdelist', axis=1) 
    return df

In [ ]:
#| eval: false
dl = delist_adj_ret(process_raw_data(get_raw_data(['dlret','dlstcd','dlstdt','exchcd','ret','siccd','naics'], nrows=1000)))   
dl.head(0)

Loading library list...
Done
Approximately 5153763 rows in crsp.msf.
Approximately 115366 rows in crsp.msenames.
Approximately 36809 rows in crsp.msedelist.
Loading library list...
Done


,,date,dtdate,dlstcd,exchcd,dlstdt,ret,naics,dlret,siccd,ret_adj
permno,Mdate,,,,,,,,,,


In [ ]:
#| export
def features(
        df: pd.DataFrame,
) -> pd.DataFrame:
    
    out = pd.DataFrame(index=df.index)
    
    out['ret_adj'] = delist_adj_ret(df, adj_ret_var='ret_adj')[['ret_adj']].copy()

    out['lbhret12'] = pdm.rrolling(1+df['ret'], window=12, func='prod') - 1
    out['retvol12'] = pdm.rrolling(df['ret'], window=12, func='std') 

    return out 

In [ ]:
#| eval: false
ftr = features(df)

In [ ]:
#| eval: false
ftr.head(1)

,,ret_adj,lbhret12,retvol12
permno,Mdate,,,
10000,1985-12,NaN,<NA>,<NA>


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()